In [1]:
import os
import cv2
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import os
from keras.models import Sequential, load_model
from tensorflow.keras.models import load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.applications.densenet import DenseNet121 as DenseNet121
from sklearn import preprocessing  #用來標準化刻度

#import matplotlib.pyplot as plt

cxr=pd.read_csv("cxr_control_mutli_ca_v2_single.csv")
count=0
train_x = [] 
test_x = [] 
train_y = [] 
test_y = [] 

for index, row in cxr.iterrows():
    if count<7132:
            img = cv2.imread("cxr_control_v2_mutli_ca" + "/" + row["dicom_id"]+".jpg")
         #  img = cv2.resize(img, (64, 64)) 
            train_x.append(img)
            train_y.append(row["eventV3"])
            count=count+1
    else :
        img = cv2.imread("cxr_control_v2_mutli_ca" + "/" + row["dicom_id"]+".jpg")
        # img = cv2.resize(img, (64, 64)) 
        test_x.append(img)
        test_y.append(row["eventV3"])
        count=count+1

train_x=np.array(train_x)
test_x=np.array(test_x)

train_y=np.array(train_y)
test_y=np.array(test_y)

#train_x = train_x.reshape(7305, 256, 256, 3)
#test_x = test_x.reshape(1502, 256, 256, 3)

train_x=train_x/255
test_x=test_x/255

print(train_x.shape)
print(test_x.shape)
#print(train_y)
#print(test_y)

(7132, 256, 256, 3)
(1452, 256, 256, 3)


In [2]:
train_x_temp=np.concatenate((train_x[:1550], train_x[-1550:]))
train_y_temp=np.concatenate((train_y[:1550], train_y[-1550:]))

print(train_x_temp.shape)
print(train_y_temp.shape)
print(train_x_temp[1])

(3100, 256, 256, 3)
(3100,)
[[[0.39607843 0.39607843 0.39607843]
  [0.15686275 0.15686275 0.15686275]
  [0.10588235 0.10588235 0.10588235]
  ...
  [0.10588235 0.10588235 0.10588235]
  [0.10588235 0.10588235 0.10588235]
  [0.10588235 0.10588235 0.10588235]]

 [[0.36862745 0.36862745 0.36862745]
  [0.14901961 0.14901961 0.14901961]
  [0.10196078 0.10196078 0.10196078]
  ...
  [0.10588235 0.10588235 0.10588235]
  [0.10588235 0.10588235 0.10588235]
  [0.10588235 0.10588235 0.10588235]]

 [[0.3254902  0.3254902  0.3254902 ]
  [0.13333333 0.13333333 0.13333333]
  [0.10196078 0.10196078 0.10196078]
  ...
  [0.10588235 0.10588235 0.10588235]
  [0.10588235 0.10588235 0.10588235]
  [0.10588235 0.10588235 0.10588235]]

 ...

 [[0.79215686 0.79215686 0.79215686]
  [0.80784314 0.80784314 0.80784314]
  [0.83137255 0.83137255 0.83137255]
  ...
  [0.08235294 0.08235294 0.08235294]
  [0.08235294 0.08235294 0.08235294]
  [0.08235294 0.08235294 0.08235294]]

 [[0.80784314 0.80784314 0.80784314]
  [0.8235

In [5]:
from tensorflow.keras.optimizers import Adam
#class_weight={0:1,1:40}
model = Sequential()
model.add(DenseNet121(include_top=False, weights="imagenet", input_shape=(256, 256, 3)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
    
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(model.summary())
history = model.fit(train_x,train_y,batch_size=8,                   
                              validation_split=0.2,
                              epochs=3)#,class_weight=class_weight
#model.save('my_model_Desnet.h5')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 8, 8, 1024)        7037504   
_________________________________________________________________
flatten_1 (Flatten)          (None, 65536)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4194368   
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 11,231,937
Trainable params: 11,148,289
Non-trainable params: 83,648
_________________________________________________________________
None
Epoch 1/3
714/714 [==============================] - 164s 169ms/step - loss: 1.5644 - accuracy: 0.

In [6]:
neur_test_y_predicted = model.predict(train_x)
predict_test=[]
for i in range(neur_test_y_predicted.shape[0]): 
    if neur_test_y_predicted[i]>0.5:
        predict_test.append(1)
    else:
        predict_test.append(0)
predict_test = np.array(predict_test)

#print(predict_test)

from sklearn.metrics import accuracy_score, f1_score,precision_score,recall_score,confusion_matrix

cm1 = confusion_matrix(train_y_temp,predict_test)

sensitivity = cm1[1,1]/(cm1[1,0]+cm1[1,1])   #TPR
specificity = cm1[0,0]/(cm1[0,0]+cm1[0,1])   #FPR

#print('Precision:',precision_score(y_test_1, predict_test))
#print('Recall:', recall_score(y_test_1, predict_test))
#print('f1-score: %f' % f1_score(y_test_1, predict_test))
print('Accuracy: %f' % accuracy_score(train_y_temp, predict_test))
print('Sensitivity : %0.2f' % sensitivity )
print('Specificity :%0.2f' % specificity)

pd.crosstab(train_y_temp,predict_test,rownames=['label'],colnames=['predict'])


MemoryError: Unable to allocate 5.22 GiB for an array with shape (7132, 256, 256, 3) and data type float32

In [ ]:
neur_test_y_predicted = model.predict(test_x)
predict_test=[]
for i in range(neur_test_y_predicted.shape[0]): 
    if neur_test_y_predicted[i]>0.5:
        predict_test.append(1)
    else:
        predict_test.append(0)
predict_test = np.array(predict_test)

#print(predict_test)

from sklearn.metrics import accuracy_score, f1_score,precision_score,recall_score,confusion_matrix

cm1 = confusion_matrix(test_y,predict_test)

sensitivity = cm1[1,1]/(cm1[1,0]+cm1[1,1])   #TPR
specificity = cm1[0,0]/(cm1[0,0]+cm1[0,1])   #FPR

#print('Precision:',precision_score(y_test_1, predict_test))
#print('Recall:', recall_score(y_test_1, predict_test))
#print('f1-score: %f' % f1_score(y_test_1, predict_test))
print('Accuracy: %f' % accuracy_score(test_y, predict_test))
print('Sensitivity : %0.2f' % sensitivity )
print('Specificity :%0.2f' % specificity)

pd.crosstab(test_y,predict_test,rownames=['label'],colnames=['predict'])
